In [1]:
import matplotlib
matplotlib.use('Agg')
# Remove padding around screen

from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import matplotlib.image as img

display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

### Simple CNN 1
The code below is a simple implementation of cnn with the same basic architecture found in ALVINN

In [2]:
from os import walk
import tensorflow as tf

import numpy
import keras
import PIL
from PIL import Image

output_angles = []
angle = -25
for i in range(45):
    output_angles.append(round(angle,2))
    angle+=1.155
    
def parse_img_name(fstr):
    data = fstr.split("_")
    if(len(data)>2):
        return(data[3])
    else:
        return -1

def prepare_files(path):
    # Numpy array containing images
    imgs = []
    # List containing steering angles
    steerings = []
    files = get_image_paths(path)
    # Prepare training data
    count = 0
    for fname in files:
        count+=1
        #if(count > 500):    # Use this to limit loaded images for testing
        #    break
        parsed = parse_img_name(fname)
        if not(parsed == -1): # Make sure the file is valid
            # steerings.append( parse_img_name(fname) ) # Append steering
            ohe_steering = one_hot_steering(parse_img_name(fname))
            if(ohe_steering == None):
                continue # Steering is not valid, continue without bad data
            steerings.append( ohe_steering )
            img = load_image(path+fname)    # Load the image
            imgs.append(img)                  # Append image to images array
    return imgs, steerings

def get_image_paths(path):
    # Walk through all the images in the folder path
    f = []
    for (dirpath, dirnames, filenames) in walk(path):
        f.extend(filenames)
    return f

def load_image(infilename):
    size = 64,60
    img = Image.open(infilename)
    img.thumbnail(size, Image.ANTIALIAS)
    data = numpy.asarray(img,dtype="uint8")
    return data

def channelSplit(image):
    return numpy.dsplit(image,image.shape[-1])

def one_hot_steering(angle):
    weights=[1,1,2,3,4,5,4,3,2,1,1]  # Weights to output feedback unit
    encoding = [0] * 45
    for i in range(len(output_angles)-1):
        if (output_angles[i] <= float(angle)) and (float(angle) <= output_angles[i+1]):
            for j in range(i-5,i+5,1): # Iterate through encoding and place weights
                if(j>=0 and j<(len(encoding))):
                    encoding[j] = weights[j+5-i]
            return encoding
        if (i == len(output_angles)-1):
            print("Data incorrectly formatted")
            return encoding
            break

/home/jason/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
output_angles[26]

5.03

In [4]:
filepath = "../datasets/track_sess_2-23/"
imgs, steerings = prepare_files(filepath)
train_y = numpy.array(steerings)
train_x = numpy.array(imgs)
# AC-frame_011922_st_23.87382_th_0.3.jpg
# one_hot_steering("24")
train_x.shape[1:]

(48, 64, 3)

In [33]:
def get_model(imgsz):
    model = keras.models.Sequential()
    #model.add(keras.layers.Dense(100, input_dim=input_size, activation='tanh'))
    model.add(keras.layers.Conv2D( 32, 5, 5, border_mode='valid', input_shape=(48, 64, 3) ))
    model.add(keras.layers.ELU())
    
    
    model.add(keras.layers.Conv2D( 16, 5, 5, border_mode='same'))
    model.add(keras.layers.ELU())
    
    model.add(keras.layers.Dropout(0.2))
    
    model.add(keras.layers.Conv2D( 16, 3, 3, border_mode='same'))
    model.add(keras.layers.ELU())
    
    model.add(keras.layers.Flatten())
    
    model.add(keras.layers.Dense(512, activation='tanh'))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.ELU())
    
    model.add(keras.layers.Dense(128, activation='tanh'))
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.ELU())
    
    model.add(keras.layers.Dense(45, activation='relu'))
    return model

# Load callbacks
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath='../outputs/track_cnnw3.h5',
    verbose=1,
    save_best_only=True)


# Get training image row, col, channels
img_sz = train_x.shape[1:]
model = get_model(img_sz)

model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 44, 60, 32)        2432      
_________________________________________________________________
elu_74 (ELU)                 (None, 44, 60, 32)        0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 44, 60, 16)        12816     
_________________________________________________________________
elu_75 (ELU)                 (None, 44, 60, 16)        0         
_________________________________________________________________
dropout_44 (Dropout)         (None, 44, 60, 16)        0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 44, 60, 16)        2320      
_________________________________________________________________
elu_76 (ELU)                 (None, 44, 60, 16)        0         
__________

/home/jason/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), input_shape=(48, 64, 3..., padding="valid")`
  after removing the cwd from sys.path.
/home/jason/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (5, 5), padding="same")`
  
/home/jason/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same")`
  del sys.path[0]


In [ ]:
model.fit(x=train_x,y=train_y,nb_epoch=40,validation_split=0.2,callbacks=[checkpointer])

/home/jason/tensorflow/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 33329 samples, validate on 8333 samples
Epoch 1/40
29824/33329 [=========================>....] - ETA: 1s - loss: 106.4179 - acc: 0.0876